In [1]:
import os
import copy
import wget
import time
import asyncio
import warnings
import logging

import numpy as np

from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS

%matplotlib inline

In [2]:
attcs = ATTCS()
cam = salobj.Remote(attcs.domain, "GenericCamera", 1)

await asyncio.gather(attcs.start_task, cam.start_task) 

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 27 messages


[[None, None, None, None, None, None, None], None]

In [6]:
# Declare target name
target_name='epsCrt'
expTime=60

In [7]:
reset_offset = {'m1': 0.0,
          'm2': 0.0,
          'x': 0. ,
          'y': 0.,
          'z': 0.,
          'u': 0.0,
          'v': 0.0
          }



In [20]:
dz = 0.8

offset_intra = {'m1': 0.0,
          'm2': 0.0,
          'x': 0.0,
          'y': 0.0,
          'z': -dz,
          'u': 0.0,
          'v': 0.0
          }

offset_extra = {'m1': 0.0,
          'm2': 0.0,
          'x': 0.0,
          'y': 0.0,
          'z': 2*dz,
          'u': 0.0,
          'v': 0.0
          }


try:
    await cam.cmd_stopLiveView.start()
except salobj.AckError:
    print("ack error")

# Intrafocal hexapod setting - offsets -0.5mm from central focus value above

print('Move to intrafocal position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                              z=hex_Z-dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(**offset_intra)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print('Take intrafocal image')
cam.evt_endReadout.flush()

try:
    await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_intra')
except Exception as e:
    await attcs.ataos.cmd_offset.set_start(z=dz)
    raise e

intra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

# Extrafocal hexapod setting
print('Move to extra-focal hexapod position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                                      z=hex_Z+dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(**offset_extra)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

# r.evt_endReadout.flush()
print('Take extra-focal image')
cam.evt_endReadout.flush()
try:
    await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_extra')
except Exception as e:
    await attcs.ataos.cmd_offset.set_start(z=-dz)
    raise e

extra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()

print(f"============")
print(f"dz={dz}")
print(intra_endron.imageName)
print(extra_endron.imageName)
print(f"============")

try:
    await cam.cmd_startLiveView.set_start(expTime=0.5)
except salobj.AckError:
    print("ack error")

attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(z=-dz)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print(f"['{intra_endron.imageName}.fits'], ['{extra_endron.imageName}.fits']")
print(np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle))

# intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
# extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'
# intra_fname = wget.download(intra_wget_url)
# print(f"============")
# print(f'{intra_fname}')
# extra_fname = wget.download(extra_wget_url)
# print(f'{extra_fname}')
# print(f"============")




Move to intrafocal position
Take intrafocal image
Move to extra-focal hexapod position
Take extra-focal image
dz=0.8
1580108605-epsCrt_intra-0-1
1580108673-epsCrt_extra-0-1
['1580108605-epsCrt_intra-0-1.fits'], ['1580108673-epsCrt_extra-0-1.fits']
227.7985615575302


In [21]:
intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'

intra_fname = wget.download(intra_wget_url, out="cwfs/data")
extra_fname = wget.download(extra_wget_url, out="cwfs/data")

print(intra_fname,extra_fname)

cwfs/data/1580108605-epsCrt_intra-0-1.fits cwfs/data/1580108673-epsCrt_extra-0-1.fits


In [19]:
offset = {'m1': 0.0,
          'm2': 0.0,
          'x': 0. ,
          'y': 0.,
          'z': 0.0538182,
          'u': 0.0,
          'v': 0.0
          }


await attcs.ataos.cmd_offset.set_start(**offset)
attcs.ataos.cmd_offset.set(**reset_offset)

# await attcs.ataos.cmd_applyAxisOffset.set_start(axis="x", offset=x_offset)
# await attcs.ataos.cmd_applyAxisOffset.set_start(axis="y", offset=y_offset)

True

In [22]:
await cam.cmd_stopLiveView.start()

In [23]:
await salobj.set_summary_state(cam, salobj.State.STANDBY)

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]